In [26]:
import os
from datetime import datetime, timedelta

import pandas as pd
import requests

from dotenv import load_dotenv
from google.cloud import storage

load_dotenv()

BOXOFFICE_API_KEY = os.getenv('BOXOFFICE_API_KEY')

def request_daily_api():
    '''
    발급받은 key와 조회하고자 하는 날짜 등 정보를 입력하여 요청해 일별 박스오피스 데이터를 가져오는 함수입니다.
    '''
    
    base_url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'
    params = {
        "key" : BOXOFFICE_API_KEY,
        "movieCd" : '20090683',           
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API 요청 실패: {response.status_code}")

def parse_daily_boxoffice_data(data):
    '''
    가져온 박스오피스 데이터 중 필요한 정보들만 모아서 Dataframe으로 변환하는 함수입니다.
    '''     
    print(data)
    movie_info = data['movieInfoResult']['movieInfo']
    
    genres = movie_info.get("genres", [])
    genre_names = ", ".join([g.get("genreNm", "") for g in genres])

    # 필요한 필드 수동으로 모음
    result = {
        "movieCd": movie_info.get("movieCd"),
        "movieNm": movie_info.get("movieNm"),
        "prdtYear": movie_info.get("prdtYear"),
        "showTm": movie_info.get("showTm"),
        "genreNm": genre_names,  # 👈 평탄화된 장르
    }

    df = pd.DataFrame([result])
    print(df)
    
data = request_daily_api()
parse_daily_boxoffice_data(data)



{'movieInfoResult': {'movieInfo': {'movieCd': '20090683', 'movieNm': '500일의 썸머', 'movieNmEn': '500 Days of Summer', 'movieNmOg': '', 'showTm': '95', 'prdtYear': '2009', 'openDt': '20100121', 'prdtStatNm': '개봉', 'typeNm': '장편', 'nations': [{'nationNm': '미국'}], 'genres': [{'genreNm': '드라마'}, {'genreNm': '멜로/로맨스'}], 'directors': [{'peopleNm': '마크 웹', 'peopleNmEn': 'Marc Webb'}], 'actors': [{'peopleNm': '조셉 고든 레빗', 'peopleNmEn': 'Joseph Gordon-Levitt', 'cast': '톰', 'castEn': ''}, {'peopleNm': '주이 디샤넬', 'peopleNmEn': 'Zooey Deschanel', 'cast': '썸머', 'castEn': ''}, {'peopleNm': '제프리 아렌드', 'peopleNmEn': 'Geoffrey Arend', 'cast': '맥켄지', 'castEn': ''}, {'peopleNm': '클로이 모레츠', 'peopleNmEn': 'Chloe Moretz', 'cast': '레이첼 핸슨', 'castEn': ''}], 'showTypes': [{'showTypeGroupNm': '필름', 'showTypeNm': '필름'}, {'showTypeGroupNm': '2D', 'showTypeNm': '디지털'}], 'companys': [{'companyCd': '20112101', 'companyNm': '스네이크 프리뷰', 'companyNmEn': 'SNEAK PREVIEW ENTERTAINMENT ', 'companyPartNm': '제작사'}, {'companyCd': 

# CGV 코드

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [27]:
driver = webdriver.Chrome()

# 사이트 접속
base_url = 'https://www.cgv.co.kr/'  # Replace with the actual URL
driver.get(base_url)

# 검색 결과 대기
driver.implicitly_wait(10)

# 검색어 입력 후 엔터
search_input = driver.find_element(By.ID,'header_keyword')
search_input.clear()
search_keyword = '하얼빈'
search_input.send_keys(search_keyword)
search_input.send_keys(Keys.RETURN)

# 검색 결과 대기
driver.implicitly_wait(10)

# 검색 결과 창을 html로 추출
results_html = driver.page_source

# 창 종료
#driver.quit()

movie_url = driver.find_element(By.ID, "searchMovieResult").find_element(By.CLASS_NAME, 'img_wrap')

print(movie_url.get_attribute("href"))
driver.get(movie_url.get_attribute("href"))

review_tab = driver.find_element(By.ID, "tabComent").find_element(By.XPATH, "a")
driver.get(review_tab.get_attribute("href"))


http://www.cgv.co.kr/movies/detail-view/?midx=88797


In [30]:
# review_link = "https://www.cgv.co.kr/movies/detail-view/?midx=88797#commentReg"

# driver.get(review_link)

# # 검색 결과 대기
# driver.implicitly_wait(10)

current_page = 1
paging = driver.find_element(By.ID, "paging_point")
page_num = paging.find_elements(By.XPATH, "./li")

cgv_reviews = []
try:
    
    while True:
        
        review_list = driver.find_element(By.ID, "movie_point_list_container")
        review_items = review_list.find_elements(By.XPATH, "./li")  # //li : 모든 li (자식, 손자 태그 포함), ./li : 자식 li

        for review in review_items:
            id = review.find_element(By.XPATH, ".//ul/li[1]").text.strip()
            date = review.find_element(By.XPATH, ".//ul/li[2]/span[1]").text.strip()
            context = review.find_element(By.XPATH, ".//div[3]/p").text.strip()
            print(id, date, context)
            
            cgv_reviews.append({
                "id" : id,
                "context" : context,
                "date" : date
            })
            
        try:
            next_page = current_page + 1
            paging = driver.find_element(By.ID, "paging_point")
            next_page_button = paging.find_element(By.XPATH, f'.//a[@href="#{next_page}"]')
            next_page_button.click()
            time.sleep(1)
            current_page += 1
            
        except:
            try:
                next_10_button = driver.find_element(By.CLASS_NAME, "btn-paging.next")
                next_10_button.click()
                time.sleep(1)
                current_page += 1
                
            except:
                break
finally:
    #driver.quit()
    pass
        
# df = pd.DataFrame(cgv_reviews)
# df.to_csv("cgv_reviews", index=False, encoding='utf-8-sig')
# print("cgv 리뷰 저장 완료")
    


fb**gml8784 2025.03.23 역사에 대해서 다루는건 좋지만 엄청 루즈하게 흘러가는 영화였습니다. 기억에 남는거 다그닥 다그닥 다그닥 밖에 없었습니다,,
xh**sql1 2025.03.22 소설 하얼빈과 뮤지컬 영웅보다 나은 부분이 뭔지 모르겠다
db**dtorl 2025.03.22 재미있게 봤던 것 같아요
뮤직바리스타 2025.03.22 아쉽다.몰입감이 넘잘생긴배우가 긴장감을 죽이는구나
구구♡♡ 2025.03.22 독립투사분들 감사합니다
wl**dss1 2025.03.22 시대를 잘 탄 영화... 화면 구성이 아름답다. 특히 전철 씬이 영상미가 죽여준다. 이동욱 수염 깎아....
CGI 2025.03.22 현빈 배우 보고 픽했던거였는데 재밌게 잘봤어요
todaysmovie 2025.03.22 영웅이 아닌 한 인간으로의 안중근
잠자는물개 2025.03.21 창작 캐릭터를 잘 만들었어요. 어느 정도 예상은 가는 스토리라인입니다.
abcd 2025.03.21 현빈으로 이거 밖에 못 뽑나요. .
선선한케빈 2025.03.21 지루..해..아는 내용이고 중요한 역사지만 이렇게 지루하게 연출할수가...
yo**07 2025.03.20 2025년 새해 첫 영화
니차브 2025.03.20 재밌게 봤습니다. ㅎ ㅎ ㅎ ㅎ
룩시우스 2025.03.20 많은 것을 뽐내려다 이도저도 못한
냠냐냐냐미 2025.03.19 넘 어두웠어요 영화관에서 배우 얼굴이 안 보일 정도.. 그래도 의미 있는 영화예요
유나나나 2025.03.19 한국 영화에서 기대 안 한 영상미
ph**ke1004 2025.03.19 연기들 나쁘지않았는데 중간중간 지루했어요
불주먹용가리 2025.03.19 광화문으로 오라는 초대장을 받았다 마음을 주셨다
하늘나대디 2025.03.18 담담하게 인물을 그린다.
hy**bc86 2025.03.18 독립운동 굿굿굿좋아요
su**0802 2025.03.18 내용을 길게 만들기 위한 노력으로 영화는 지루해짐
김은지꽃 2025.03.18 에국심이감동이게 잘봐서요
한

KeyboardInterrupt: 

# 씨네 21 코드

In [18]:
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

# 사이트 접속
base_url = 'http://www.cine21.com/'  # Replace with the actual URL
driver.get(base_url)

# 검색 결과 대기
driver.implicitly_wait(10)

# 검색어 입력 후 엔터
search_input = driver.find_element(By.CSS_SELECTOR,'#search_q.input_search')
search_input.clear()
search_keyword = '하얼빈'
search_input.send_keys(search_keyword)
search_input.send_keys(Keys.RETURN)

# 검색 결과 대기
driver.implicitly_wait(10)

cine_reviews = []

movie_link = driver.find_element(By.CLASS_NAME, "mov_list").find_element(By.TAG_NAME, "a")
driver.get(movie_link.get_attribute("href"))
driver.implicitly_wait(10)

results_html = driver.page_source

soup = BeautifulSoup(results_html, 'html.parser')

try:
    expert_rating = soup.find('ul', 'expert_rating')
    expert_reviews = expert_rating.find_all('li')

    for review in expert_reviews:
        star = review.select_one("div > div.star_area > span")
        name = review.select_one("div > div.comment_area > a > span")
        context = review.select_one("div > div.comment_area > span")
        
        cine_reviews.append({
            "who" : "expert",
            "name" : name,
            "context" : context,
            "star" : star,
            "date" : ""
        })
except:
    pass    # 전문가 리뷰 찾을 수 없을 때

try:    
    # 네티즌 리뷰있는 곳으로 스크롤
    netizen_review_area = driver.find_element(By.ID, "netizen_review_area")
    driver.execute_script("arguments[0].scrollIntoView(true);", netizen_review_area)
    time.sleep(1)
    
    pagination = netizen_review_area.find_element(By.CLASS_NAME, "pagination")      # page 
    page_buttons = pagination.find_elements(By.CSS_SELECTOR, ".page > a")
    last_page = int(page_buttons[-1].text.strip())  # 마지막 페이지
    
    for page in range(1, last_page + 1):
        driver.execute_script("$('#netizen_review_area').nzreview('list', arguments[0]);", page)
        # driver.implicitly_wait(10)
        
        netizen_review_area = driver.find_element(By.ID, "netizen_review_area")
        driver.execute_script("arguments[0].scrollIntoView(true);", netizen_review_area)
        # driver.implicitly_wait(10)
    
        # 페이지 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "reply_box"))
        )
        time.sleep(1)  # 추가 대기 (렌더링 안정화)
        
        netizen_review_list = netizen_review_area.find_element(By.CLASS_NAME, 'reply_box')  # 리뷰 목록들
        review_items = netizen_review_list.find_elements(By.XPATH, "./li")

        try:
            for review in review_items:
                name = review.find_element(By.CLASS_NAME, "id").text.strip()
                date = review.find_element(By.CLASS_NAME, "date").text.split()
                star = review.find_element(By.XPATH, "./div[3]/span").text.strip()
                try:
                    context = review.find_element(By.CSS_SELECTOR, "div.comment.ellipsis_3").text.strip()
                except:
                    try:
                        context = review.find_element(By.CSS_SELECTOR, "div.comment").text.strip()
                    except Exception as e:
                        print(f"Error {e} : Netizen context crawling")
                
                cine_reviews.append({
                    "who" : "netizen",
                    "name" : name,
                    "context" : context,
                    "star" : star,
                    "date" : date[0]
                })
                print(name, context, star, date[0])
        except Exception as e:
            print(f"Error {e}")
            continue
except Exception as e:
    print("Netizen review 없음")
    print(f"Error {e}")
        
        
    
    


186***** 시대착오적인 반일 국뽕 영화. 정말 영화계 좌파들 반일타령 지긋지긋하다. 이미 한해에 881만명의 한국인들이 일본을 찾는다는데.. 아직도 100년전 제국주의 시대를 들먹이며 멍청한 선조들의 치욕스런 역사를 빨아대고 있으니원. 배우들 연기도 바닥을 치고. 충분히 망할만 하다고 생각한다. 조폭 영화보다 더 지긋지긋함. 1 2025-03-15
389***** 내가 본 것중 최고 수준의 영화~!!!
시간이 어떻게 갔는지 모르겠어요 10 2025-01-24
831***** 각종 비평도 좋지만 감성적으로라도 독립운동가의 숭고한 희생을 최소한으로 기억하는 것이 후세로서의 도리라고 생각한다.
독립운동 얼마나 힘들었겠는가.. 10 2025-01-23
257***** 이상하네요 분명히 별점이 낮았었구 지루하고 현빈 발연기에 대한 혹평이 많아서 안봤었는데 갑자기 별점이 높아져서 혹시나 하고 가서 봤더니 정말 더럽게 재미 없네요 솔직평 남기려고 로그인 하는데 계속 에러나서 별점 조작의심이 많이 듭니다 절대 비추 주변사람들도 자는 사람이 많네요 1 2025-01-20
607***** 중근이는 약속을 지켰다. 8 2025-01-06
386***** 영화가 끝나도 눈물이...너무 가슴 아프고 감사했습니다. 딸에게 나라의 소중함을 잊지말고 살아야한다고 당부했습니다. 10 2025-01-04
386***** 압도적인 영상미와 한민족 역사가 계속 존재하는 이유가 여기 있네요
아내와 같이 새해 첫 영화로 후회 없는 선택이였네요!! 추천합니다. 10 2025-01-02
354***** 최근 극장에서 본 영화 중 가장 완성도가 높았던 영화. 지금 우리가 이 땅에 살 수 있게 해준 그들에게 감사하며..! 10 2025-01-02
114***** 좀전에 아내와 함께 강동 롯데에서 보고 왔습니다. 이 시국에 꼭 봐야할 영화, 안중근 의사의 희생과 사랑과 동지애 또 그와 함께한 많은 이들의 희생으로 일구어진 거사.. 우리나라에 이런 리더가 나올 수 있기를 바라는 마음 간절하네요. 이토오히

# 왓챠피디아 코드

In [21]:
from google.cloud import storage
from datetime import datetime, timedelta
import os, io
from dotenv import load_dotenv

load_dotenv()

GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
BUCKET_NAME = os.getenv('BUCKET_NAME')
MOVIE_REVIEW_FOLDER = 'movie_reviews'
DAILY_BOXOFFICE_FOLDER = 'daily_boxoffice'
DAILY_REGION_BOXOFFICE_FOLDER = 'daily_regions_boxoffice'
BOXOFFICE_API_KEY = os.getenv('BOXOFFICE_API_KEY')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

def get_unique_movie_list_from_gcs():
    client = storage.Client()
    bucket = client.bucket(BUCKET_NAME)
    
    # 중복 없이 영화 리스트 담기 위함
    movie_set = set()
    
    def process_boxoffice(folder):
        blobs = bucket.list_blobs(prefix=folder)
        for blob in blobs:
            if blob.name.endswith(".csv"):  # csv파일에 담긴 boxoffice파일
                csv_data = blob.download_as_text(encoding='utf-8-sig')
                df = pd.read_csv(io.StringIO(csv_data))
                
                if 'movieNm' in df.columns and 'openDt' in df.columns:
                    for _,row in df.iterrows():
                        movieNm = str(row["movieNm"]).strip()
                        openDt = str(row["openDt"]).strip()
                        movie_set.add((movieNm, openDt))
    
    process_boxoffice(DAILY_BOXOFFICE_FOLDER)   # 일별 박스오피스 조회
    process_boxoffice(DAILY_REGION_BOXOFFICE_FOLDER)    # 지역별 박스오피스 조회    
    
    return movie_set
movie = get_unique_movie_list_from_gcs()
print(len(movie))

55


In [22]:
def crawling_cgv_review():

    # for movieNm, openDt in movie_info_set:
    #     if movieNm in ('모아나 2', '대가족', '괜찮아 괜찮아 괜찮아!', '엘리: 몬스터 패밀리', '폭락', '9월 5일: 위험한 특종', '노스페라투', '데드데드 데몬즈 디디디디 디스트럭션: 파트1', 
    #                    '브루탈리스트', '시빌 워: 분열의 시대', '페라리','보고타: 마지막 기회의 땅', '검은 수녀들', '그 시절, 우리가 좋아했던 소녀','극장판 쿠로코의 농구 라스트 게임',
    #                    '러브레터', '하얼빈', '수퍼 소닉3', '데드데드 데몬즈 디디디디 디스트럭션: 파트2', '영화 이상한 과자 가게 전천당',
    #                    '더 폴: 디렉터스 컷', '백수아파트', '브로큰', '써니데이', '엘리: 몬스터 패밀리', '캡틴 아메리카: 브레이브 뉴 월드', '컴플리트 언노운', '퇴마록'):
    #         continue
        cgv_reviews = []
        movieNm = '위키드'
        driver = webdriver.Chrome()

        # 사이트 접속
        url = 'http://www.cgv.co.kr/movies/detail-view/?midx=88076'
        driver.get(url)

        # 검색 결과 대기
        driver.implicitly_wait(10)
        
        try:
            # 영화 제목 입력 후 엔터
            # search_input = driver.find_element(By.ID,'header_keyword')
            # search_input.clear()
            # search_input.send_keys(movieNm)
            # search_input.send_keys(Keys.RETURN)

            # # 검색 결과 대기
            # driver.implicitly_wait(10)
            # time.sleep(1)

            # # 영화 상세 url
            # movie_url = driver.find_element(By.ID, "searchMovieResult").find_element(By.CLASS_NAME, 'img_wrap')
            # driver.get(movie_url.get_attribute("href"))

            # 영화 평점/리뷰로 이동할 수 있는 탭
            review_tab = driver.find_element(By.ID, "tabComent").find_element(By.XPATH, "a")
            driver.get(review_tab.get_attribute("href"))
            
            # 현재 페이지 1 페이지
            current_page = 1
            paging = driver.find_element(By.ID, "paging_point")
            
            while True:
                
                review_list = driver.find_element(By.ID, "movie_point_list_container")
                review_items = review_list.find_elements(By.XPATH, "./li")  # //li : 모든 li (자식, 손자 태그 포함), ./li : 자식 li
                
                if not review_items:
                    break

                # for review in review_items:
                for i in range(len(review_items)):
                    try:
                        review_list = driver.find_element(By.ID, "movie_point_list_container")
                        review_items = review_list.find_elements(By.XPATH, "./li")
                        review = review_items[i]
                        
                        id = review.find_element(By.XPATH, ".//ul/li[1]").text.strip()
                        date = review.find_element(By.XPATH, ".//ul/li[2]/span[1]").text.strip()
                        context = review.find_element(By.XPATH, ".//div[3]/p").text.strip()
                        print(id, date, context)
                        
                        cgv_reviews.append({
                            "id" : id,
                            "context" : context,
                            "date" : date
                        })
                    except Exception as e:
                        continue
                    
                try:    # 다음 페이지 있을 때 
                    next_page = current_page + 1
                    paging = driver.find_element(By.ID, "paging_point")
                    next_page_button = paging.find_element(By.XPATH, f'.//a[@href="#{next_page}"]')
                    next_page_button.click()
                    time.sleep(1)
                    current_page += 1
                    
                except:
                    try:   
                        next_10_button = driver.find_element(By.CLASS_NAME, "btn-paging.next")
                        next_10_button.click()
                        time.sleep(1)
                        current_page += 1
                        
                    except:
                        break
        except Exception as e:
            print("cgv에서 상영한 영화가 아니거나 리뷰 수집을 못함")
        finally:
            driver.quit()
            
            if cgv_reviews:
                df = pd.DataFrame(cgv_reviews)
                upload_to_gcs(df, movieNm)
            else:
                print(f"{movieNm} 리뷰 없음")

def upload_to_gcs(df, movieNm):
    client = storage.Client()
    bucket = client.bucket(BUCKET_NAME)
    
    # gcs 파일 경로 설정
    gcs_file_path = f"{MOVIE_REVIEW_FOLDER}/cgv_reviews/{movieNm}_cgv_reviews.csv"
    blob = bucket.blob(gcs_file_path)
    
    csv_data = df.to_csv(index=False, encoding='utf-8-sig')
    blob.upload_from_string(csv_data, content_type="text/csv")    
    
    print(f"cgv reviews 업로드 완료. 날짜 : {movieNm}")
    
crawling_cgv_review()

dk**kels1004 2025.03.25 몰아치는 음악과 꿈같이 그려낸 동화 속 그 곳 살짝 틀어낸 스토리
41**sther 2025.03.25 뉴욕에서 본 뮤지컬이 생각나서 좋았어요
광수긁는유재석 2025.03.24 재밌음. 다음편을 달라.
ki**w9905 2025.03.23 재미있게 잘 봤습니다. 감사합니다
사악한은단 2025.03.22 기대이상의 작품입니드
ru**d12 2025.03.22 원작을 모르고 봤는데 교내 퀸카만들기?만 걷어냈어도... 아 파퓰러 때문인가?
CGI 2025.03.22 주연 두분의 환상 콤비덕에 재밌었어요
kk**t0510 2025.03.21 춤과 음악을 보는 재미가 있었어요
Hoker 2025.03.21 생각보다 이야기가 감동적으로 잘 와닿는데, 대놓고 파트 2 보세요 라고 시작부터 깔면 김이 새지… 그래도 마무리는 깔끔하게 지어줘서 다행.
pi**gu7979 2025.03.21 영화관에서 보기를 잘 했다고 생각한 영화
ca**t0524 2025.03.21 더빙이랑 자막 둘다 봤는데 너무 재밌었어요
0s**rah 2025.03.21 You won't regret watching it.
오트 2025.03.21 아리 글린다 너무 사랑스러워 ♥
yo**07 2025.03.20 더빙이라 당황했지만 그럼에도 불구하고 몰입감이 좋았던 영화
gj**s8388 2025.03.20 최고였음 최근에 나온 영화중
영화광 남 2025.03.19 너무 재미없다 스토리가 뮤지컬이 인상적이고 개성적이지가 않았다
ev**s83 2025.03.19 아리아나그란데가 노래를 잘 부르더라고요ㅎㅎ
쏠ㄹㅏC 2025.03.19 둘이서 행복하게 살게 해죠요ㅠ 둘을 갈라놓지 말란말이어요ㅠㅠ
네천사별 2025.03.19 즐거운 영화 뮤지컬 익숙한 노래들 즐거웠다
영민이남편 2025.03.19 오랜만에 볼만한 뮤지컬 무비였네요~ 애들이랑 2편도 보러갈 예정입니다!!
ky**1230 2025.03.19 알록달록 재밌는 영화!
루 2025.03.19 얼른 2편이 나왓으면 